In [ ]:
# Install Airavata SDK with Jupyter support
%pip install -qU "airavata-python-sdk[notebook]==2.2.3"

# Import Airavata Jupyter magics for managing remote HPC jobs
import airavata_jupyter_magic

# Authenticate the session (prompts for credentials or uses tokens)
%authenticate

# Request an HPC runtime with GPU support using a YAML job config
%request_runtime hpc_gpu --file=cybershuttle.yml --walltime=130 --use=AnvilGPU:gpu

# Wait for the runtime to be ready
%wait_for_runtime hpc_gpu --live

# Switch to the requested HPC runtime
%switch_runtime hpc_gpu

In [ ]:
%%bash
set echo on 
# clone the attention visualization demo repo into the workspace
git clone \
  https://github.com/vizfold/attention-viz-demo \
  attention-viz-demo

# cd to attention-viz-demo directory
cd attention-viz-demo

# Download required files for OpenFold from external sources
wget -N --no-check-certificate \
  -P openfold/resources \
  https://git.scicore.unibas.ch/schwede/openstructure/-/raw/7102c63615b64735c4941278d92b554ec94415f8/modules/mol/alg/src/stereo_chemical_props.txt

date 
echo "Installing openfold params"
# Download pretrained OpenFold model weights
./scripts/download_openfold_params.sh openfold/resources/params

date
# Create output directory to save results
mkdir -p outputs

echo "Installing openfold tools "
# Set up openfold tools
pip install -e .

date

In [ ]:
# Required imports
import os
import subprocess

# Define target protein and the residue to center triangle attention on
PROT = "6KWC"
TRI_RESIDUE_IDX = 18

# Define all relevant directories
BASE_DATA_DIR = "/depot/itap/datasets/alphafold/db"  # path to AlphaFold2 data
ATTN_MAP_DIR = f"./outputs/attention_files_{PROT}_demo_tri_{TRI_RESIDUE_IDX}" # directory for saving text files with top-k attention scores
ALIGNMENT_DIR = "./examples/monomer/alignments" # directory containing pre-computed alignment files (and MSAs)
OUTPUT_DIR = f"./outputs/my_outputs_align_{PROT}_demo_tri_{TRI_RESIDUE_IDX}" # directory to save outputs
IMAGE_OUTPUT_DIR = f"./outputs/attention_images_{PROT}_demo_tri_{TRI_RESIDUE_IDX}"
FASTA_DIR = f"/u/thayes/vizfold/examples/monomer/fasta_dir_{PROT}"

# Note: If this is a new protein, the ALIGNMENT_DIR does not need to be specified here or in the next cell
# In this case, the code will compute MSAs and alignments, which can take several hours


In [ ]:
# Run OpenFold inference and save top attention scores to text files 
inference_cmd = f"""
python3 run_pretrained_openfold.py \
    {FASTA_DIR} \
    {BASE_DATA_DIR}/pdb_mmcif/mmcif_files \
    --use_precomputed_alignments {ALIGNMENT_DIR} \
    --output_dir {OUTPUT_DIR} \
    --config_preset model_1_ptm \
    --uniref90_database_path {BASE_DATA_DIR}/uniref90/uniref90.fasta \
    --mgnify_database_path {BASE_DATA_DIR}/mgnify/mgy_clusters_2022_05.fa \
    --pdb70_database_path {BASE_DATA_DIR}/pdb70/pdb70 \
    --uniclust30_database_path {BASE_DATA_DIR}/uniclust30/uniclust30_2018_08 \
    --bfd_database_path {BASE_DATA_DIR}/bfd/bfd_metaclust_clu_complete_id30_c90_final_seq.sorted_opt \
    --save_outputs \
    --model_device "cuda:0" \
    --attn_map_dir {ATTN_MAP_DIR} \
    --num_recycles_save 1 \
    --triangle_residue_idx {TRI_RESIDUE_IDX} \
    --demo_attn
"""

subprocess.run(inference_cmd, shell=True, check=True)


In [ ]:
# Render predicted 3D structure and save as PNG image
from visualize_attention_general_utils import render_pdb_to_image

PDB_FILE = os.path.join(OUTPUT_DIR, f"predictions/{PROT}_1_model_1_ptm_relaxed.pdb")
FNAME = f"predicted_structure_{PROT}_tri_{TRI_RESIDUE_IDX}.png"

render_pdb_to_image(PDB_FILE, IMAGE_OUTPUT_DIR, FNAME)


In [ ]:
# Import visualization utilities
from visualize_attention_3d_demo_utils import plot_pymol_attention_heads
from visualize_attention_arc_diagram_demo_utils import generate_arc_diagrams, parse_fasta_sequence

# Setup visualization output directories
output_dir_msa = os.path.join(IMAGE_OUTPUT_DIR, 'msa_row_attention_plots') # directory for saving msa attention 3D visuals
output_dir_tri = os.path.join(IMAGE_OUTPUT_DIR, 'tri_start_attention_plots') # directory for saving triangle attention 3D visuals
FASTA_PATH = f"/u/thayes/vizfold/examples/monomer/fasta_dir_{PROT}/{PROT}.fasta"
LAYER_IDX = 47 # selected layer for attention evaluation
TOP_K = 50 # show top-k attention links (limit to 500)

# Generate 3D attention plots for MSA row attention
plot_pymol_attention_heads(
    pdb_file=PDB_FILE,
    attention_dir=ATTN_MAP_DIR,
    output_dir=output_dir_msa,
    protein=PROT,
    attention_type="msa_row",
    top_k=TOP_K,
    layer_idx=LAYER_IDX
)

# Generate 3D attention plots for triangle start attention
plot_pymol_attention_heads(
    pdb_file=PDB_FILE,
    attention_dir=ATTN_MAP_DIR,
    output_dir=output_dir_tri,
    protein=PROT,
    attention_type="triangle_start",
    residue_indices=[TRI_RESIDUE_IDX],
    top_k=TOP_K,
    layer_idx=LAYER_IDX
)

# Parse FASTA for arc diagrams
residue_seq = parse_fasta_sequence(FASTA_PATH)

# Generate arc diagrams for MSA row attention
generate_arc_diagrams(
    attention_dir=ATTN_MAP_DIR,
    residue_sequence=residue_seq,
    output_dir=output_dir_msa,
    protein=PROT,
    attention_type="msa_row",
    top_k=TOP_K,
    layer_idx=LAYER_IDX
)

# Generate arc diagrams for triangle start attention
generate_arc_diagrams(
    attention_dir=ATTN_MAP_DIR,
    residue_sequence=residue_seq,
    output_dir=output_dir_tri,
    protein=PROT,
    attention_type="triangle_start",
    residue_indices=[TRI_RESIDUE_IDX],
    top_k=TOP_K,
    layer_idx=LAYER_IDX
)


In [ ]:
# Import function for combining attention plots
from visualize_attention_general_utils import generate_combined_attention_panels

# Combine MSA row plots
generate_combined_attention_panels(
    attention_type="msa_row",
    protein=PROT,
    layer_idx=LAYER_IDX,
    output_dir_3d=output_dir_msa,
    output_dir_arc=output_dir_msa,
    combined_output_dir=IMAGE_OUTPUT_DIR,
)

# Combine triangle start plots
generate_combined_attention_panels(
    attention_type="triangle_start",
    protein=PROT,
    layer_idx=LAYER_IDX,
    output_dir_3d=output_dir_tri,
    output_dir_arc=output_dir_tri,
    combined_output_dir=IMAGE_OUTPUT_DIR,
    residue_indices=[TRI_RESIDUE_IDX]
)
